In [15]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

In [16]:
#Generating prior probability thresholds
# Flatten all genre lists into one long list
all_genres = sorted(set(genre for sublist in df_train["genres"] for genre in sublist))

# Flatten the genre lists
all_genres_comb = [genre for genre_list in df_train['genres'] for genre in genre_list]

# Count each genre
genre_counts = Counter(all_genres_comb)

# Total number of samples
total_samples = len(df_train)

# Compute prior probability for each genre
genre_priors = {genre: count / total_samples for genre, count in genre_counts.items()}

# Display
prior_threshold=[]
for genre, prob in sorted(genre_priors.items()):
    print(f"{genre}: {prob:.4f}")
    prior_threshold.append(prob)
prior_threshold

Action: 0.3128
Adventure: 0.2180
Comedy: 0.4091
Drama: 0.2069
Fantasy: 0.2368
Historical: 0.0743
Kids: 0.0834
Mecha: 0.0763
Music: 0.0514
Mystery: 0.0666
Romance: 0.1678
School: 0.1360
Sci-Fi: 0.1993
Seinen: 0.0712
Shoujo: 0.0565
Shounen: 0.1603
Slice of Life: 0.1410
Sports: 0.0507
Super Power: 0.0500
Supernatural: 0.1277


[0.3127808988764045,
 0.21797752808988763,
 0.40912921348314607,
 0.20688202247191012,
 0.23679775280898877,
 0.07429775280898876,
 0.08342696629213484,
 0.07626404494382022,
 0.05140449438202247,
 0.06657303370786517,
 0.1678370786516854,
 0.13595505617977527,
 0.19929775280898876,
 0.07120786516853933,
 0.056460674157303374,
 0.16025280898876404,
 0.14101123595505619,
 0.050702247191011234,
 0.05,
 0.12766853932584268]

In [17]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef

def optimize_thresholds_with_precision_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000), min_precision=0.5):
    best_thresholds = []
    for i in range(probs.shape[1]):
        best_f1 = 0
        best_thresh = 0.5
        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            precision = precision_score(labels[:, i], pred, zero_division=0)
            f1 = f1_score(labels[:, i], pred, zero_division=0)
            if f1 > best_f1 and precision >= min_precision:
                best_f1 = f1
                best_thresh = t
        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, F1 = {best_f1:.4f}")
    return (best_thresholds)

def compute_tpr_tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return tpr, tnr

def optimize_thresholds_with_mcc_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000)):
    best_thresholds = []

    for i in range(probs.shape[1]):
        best_mcc = -1
        best_thresh = 0.5

        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            tpr, tnr = compute_tpr_tnr(labels[:, i], pred)
            mcc = matthews_corrcoef(labels[:, i], pred)
            if mcc > best_mcc:
                best_mcc = mcc
                best_thresh = t
 

        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, MCC = {best_mcc:.4f}")
    
    return best_thresholds

#optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
#optimised_thresh=optimize_thresholds_with_mcc_constraint(val_prob,val_labels)

In [18]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting


# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
#display(y)
vectorizer = CountVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4),min_df=2,max_df=0.8)
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

classifier = OneVsRestClassifier(MultinomialNB(alpha=0.1))#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)  

OneVsRestClassifier(estimator=MultinomialNB(alpha=0.1))

In [32]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency:")
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Combine words with their class-specific frequencies
    word_freq_pairs = list(zip(feature_names, class_counts))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_freq_pairs, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies
    print("Top 10 words with highest frequency:")
    for word, count in sorted_words[:20]:
        print(f"{word}: {count}")
        
    print("\n" + "=" * 50 + "\n")



Class 'Action' word frequency:
Top 10 words with highest frequency:
world: 1036
new: 607
life: 602
year: 550
power: 454
friend: 442
girl: 422
school: 413
human: 408
mysterious: 401
earth: 400
battle: 399
city: 383
war: 372
day: 362
young: 354
known: 349
time: 332
fight: 323
story: 301


Class 'Adventure' word frequency:
Top 10 words with highest frequency:
world: 824
life: 414
new: 410
friend: 367
year: 350
power: 315
earth: 289
time: 282
young: 273
girl: 271
mysterious: 271
human: 256
named: 255
day: 242
story: 240
boy: 237
way: 226
journey: 218
city: 215
people: 213


Class 'Comedy' word frequency:
Top 10 words with highest frequency:
school: 1015
girl: 915
life: 796
world: 780
new: 779
friend: 683
day: 579
time: 498
student: 464
episode: 457
high: 457
year: 449
story: 436
love: 377
high school: 375
make: 369
series: 366
way: 345
family: 325
boy: 306


Class 'Drama' word frequency:
Top 10 words with highest frequency:
life: 610
school: 497
girl: 470
world: 463
year: 422
friend: 405
n

In [8]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency and probability:")

    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class (already calculated previously)
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies and their probabilities
    print("Top 10 words with highest frequency and probability:")
    print(f"{'Word':<20} {'Frequency':<10} {'Probability':<15}")
    print("=" * 50)
    for word, count, probability in sorted_words[:10]:
        print(f"{word:<20} {count:<10} {probability:.4f}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
world                1036       0.0061
new                  607        0.0058
life                 602        0.0071
year                 550        0.0042
power                454        0.0022
friend               442        0.0053
girl                 422        0.0079
school               413        0.0080
human                408        0.0027
mysterious           401        0.0023


Class 'Adventure' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
world                824        0.0060
life                 414        0.0067
new                  410        0.0057
friend               367        0.0048
year                 350        0.0044
power                315        0.0024
earth                289        0.0020
time                 282        0.00

In [61]:
# Step 1: Input Text Preprocessing
sample_text = "pilot battle planet robot fight in space"
print(f"Sample Text: {sample_text}")

# Vectorize the input text using the same vectorizer used for training
X_sample = vectorizer.transform([sample_text])
pred=classifier.predict_proba(X_sample)
print("predicted probabilities")
for i in range(0,len(pred[0].tolist())):
    print(f"{all_genres[i]}={pred[0][i]}")
# Step 2: Mapping Features
print("\nStep 2: Mapping Features")
present_word_indices = X_sample.nonzero()[1]  # Get indices of non-zero features
feature_names = vectorizer.get_feature_names_out()

print(f"Words present in the sample:")
for index in present_word_indices:
    print(f"- {feature_names[index]} (Index: {index})")

# Step 3: Extracting Model Parameters
print("\nStep 3: Extracting Model Parameters")
for class_index, estimator in enumerate(classifier.estimators_):
    class_name = mlb.classes_[class_index]
    print(f"\nClass: {class_name}")

    # Get the prior probability for the class
    prior_log = estimator.class_log_prior_[0]
    prior = np.exp(prior_log)
    print(f"Log Prior: {prior_log:.4f}, Prior Probability: {prior:.4f}")

    # Get the log probabilities for words in the class
    log_prob = estimator.feature_log_prob_[0]

    # Step 4: Calculating Log Probabilities
    log_prob_class = prior_log  # Start with the log prior

    print("\nCalculating log probability:")
    for index in present_word_indices:
        word = feature_names[index]
        word_log_prob = log_prob[index]
        log_prob_class += word_log_prob
        print(f"Word: {word:<20} Log P(word|{class_name}): {word_log_prob:.4f}")

    # Step 5: Exponentiation to Get Final Probability
    final_prob = np.exp(log_prob_class)
    print(f"\nLog Sum for {class_name}: {log_prob_class:.4f}")
    print(f"Final Probability for {class_name}: {final_prob:.4f}")


Sample Text: pilot battle planet robot fight in space
predicted probabilities
Action=0.9782900665570801
Adventure=0.7902130161703154
Comedy=0.045823235119317275
Drama=0.11854035354265431
Fantasy=0.008760036390328992
Historical=0.00033741773093282144
Kids=0.016330815543897998
Mecha=0.9996635088045812
Music=0.0004826158807966897
Mystery=6.392538670231276e-05
Romance=0.005021570394863842
School=0.0001289826277087438
Sci-Fi=0.9999339752660713
Seinen=0.006037486045298855
Shoujo=9.257094658479882e-07
Shounen=0.17990872489299573
Slice of Life=3.388315669380318e-05
Sports=0.0008339333859334194
Super Power=0.007333325875266134
Supernatural=5.230719980664343e-05

Step 2: Mapping Features
Words present in the sample:
- battle (Index: 764)
- fight (Index: 3041)
- pilot (Index: 6445)
- planet (Index: 6481)
- robot (Index: 7307)
- space (Index: 8165)

Step 3: Extracting Model Parameters

Class: Action
Log Prior: -0.3751, Prior Probability: 0.6872

Calculating log probability:
Word: battle           

In [9]:
# List of specific words to check
specific_words = ["love", "action", "friend", "school", "battle"]  # Modify this list as needed

# Loop through each class within the OneVsRestClassifier
for word in specific_words:
    print(f"Word: '{word}'")
    print(f"{'Class':<20} {'Probability':<15}")
    print("=" * 50)

    # Find the index of the word in the vocabulary
    try:
        word_index = vectorizer.vocabulary_[word]
    except KeyError:
        print(f"Word '{word}' not found in vocabulary.")
        print("=" * 50 + "\n")
        continue

    # Loop through each class and print the word probability
    for class_index, estimator in enumerate(classifier.estimators_):
        # Get the class name
        class_name = mlb.classes_[class_index]

        # Extract the log probabilities from the Naive Bayes model
        log_prob = estimator.feature_log_prob_

        # Convert log probabilities to normal probabilities
        prob = np.exp(log_prob)

        # Get the probability of the word given the class
        word_prob = prob[0, word_index]
        word_log_prob=log_prob[0, word_index]

        # Print the class name and word probability
        print(f"{class_name:<20} {word_log_prob:.4f}")

    print("=" * 50 + "\n")


Word: 'love'
Class                Probability    
Action               -5.6674
Adventure            -5.8050
Comedy               -6.0981
Drama                -6.0985
Fantasy              -5.8656
Historical           -5.9621
Kids                 -5.9321
Mecha                -5.9139
Music                -5.9720
Mystery              -5.9131
Romance              -6.4731
School               -6.1354
Sci-Fi               -5.8399
Seinen               -5.9624
Shoujo               -6.0716
Shounen              -5.8965
Slice of Life        -6.0563
Sports               -5.9495
Super Power          -5.9293
Supernatural         -5.9035

Word: 'action'
Class                Probability    
Action               -7.8592
Adventure            -7.6469
Comedy               -7.6566
Drama                -7.5661
Fantasy              -7.5080
Historical           -7.5818
Kids                 -7.5656
Mecha                -7.6142
Music                -7.5641
Mystery              -7.5597
Romance              -7.543

In [10]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency and probability:")

    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class (already calculated previously)
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[2], reverse=True)

    # Display the top 10 words with the highest frequencies and their probabilities
    print("Top 10 words with highest frequency and probability:")
    print(f"{'Word':<20} {'Frequency':<10} {'Probability':<15}")
    print("=" * 50)
    for word, count, probability in sorted_words[20:40]:
        print(f"{word:<20} {count:<10} {probability:.4f}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
series               225        0.0026
family               198        0.0026
way                  290        0.0026
help                 207        0.0025
come                 240        0.0024
club                 43         0.0023
named                275        0.0023
people               280        0.0023
mysterious           401        0.0023
meet                 201        0.0022
anime                131        0.0022
power                454        0.0022
team                 233        0.0021
work                 158        0.0021
home                 154        0.0021
character            121        0.0021
father               189        0.0020
game                 254        0.0020
set                  252        0.0020
end                  220        0.0020


Class 'Adventure' word frequency and probability:
Top 10 words with hi

In [25]:
import pandas as pd
import numpy as np

# Create an empty dictionary to store genre and top words
genre_words = {"Genre": [], "Top Word 1": [], "Top Word 2": [], "Top Word 3": [], "Top Word 4": [], "Top Word 5": []}

# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    # Get the genre name
    genre_name = mlb.classes_[class_index]
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()
    
    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_
    
    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)
    
    # Combine words with their probabilities
    word_stats = list(zip(feature_names, prob[0]))
    
    # Sort words by probability in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)[:5]
    
    # Extract the top 5 words for this genre
    top_words = [word for word, _ in sorted_words]
    
    # Store in the dictionary
    genre_words["Genre"].append(genre_name)
    for i in range(5):
        genre_words[f"Top Word {i+1}"].append(top_words[i] if i < len(top_words) else '')

# Convert to DataFrame
df_top_words = pd.DataFrame(genre_words)

# Display the DataFrame
display(df_top_words)
df_top_words.to_csv("words BOWMNB.csv")


,Genre,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5
0,Action,school,girl,life,world,new
1,Adventure,school,girl,life,world,new
2,Comedy,world,life,girl,school,new
3,Drama,world,girl,school,life,new
4,Fantasy,school,girl,life,new,world
5,Historical,world,school,girl,life,new
6,Kids,world,school,girl,life,new
7,Mecha,world,school,girl,life,new
8,Music,world,life,school,girl,new
9,Mystery,world,girl,school,life,new


In [13]:
import pandas as pd
import numpy as np

# Initialize the DataFrame with column names
influential_words_df = pd.DataFrame(columns=[f"Top Word {i+1}" for i in range(5)])

for class_index, estimator in enumerate(classifier.estimators_):
    genre_name = mlb.classes_[class_index]
    
    # Get feature names and log probabilities
    feature_names = vectorizer.get_feature_names_out()
    log_prob = estimator.feature_log_prob_

    # Calculate log odds (difference between current class and the average of others)
    log_odds = log_prob[0] - np.mean(log_prob, axis=0)

    # Combine words with their log-odds
    word_stats = list(zip(feature_names, log_odds))

    # Sort words by absolute log-odds in descending order
    sorted_words = sorted(word_stats, key=lambda x: abs(x[1]), reverse=True)

    # Select the top 5 most influential words
    top_5_words = [f"{word} ({log_odds:.4f}) " for word, log_odds in sorted_words[:5]]#({log_odds:.4f})

    # Add the top words to the DataFrame as a new row
    influential_words_df.loc[genre_name] = pd.Series(top_5_words, index=influential_words_df.columns)

# Display the DataFrame
display(influential_words_df)

#influential_words_df.to_csv('MNB words.csv',  index=False)


,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5
Action,gokuu (-3.2990),kirito (-3.0109),admiral (-2.9887),gunpla (-2.9158),kamui (-2.9158)
Adventure,gohan (-3.2386),kuiba (-3.2131),sinbad (-3.2131),eureka (-3.1578),pero (-3.1578)
Comedy,doraemon (-3.1266),tenchi (-3.0964),hilarious (-3.0805),comedic (-3.0473),gintoki (-3.0298)
Drama,karasuno (-3.3013),kircheis (-3.0675),takato (-3.0675),vale (-2.9965),hiroki (-2.9568)
Fantasy,precure (-3.4538),kirito (-3.2590),kuiba (-3.1639),bakugan (-3.0785),rimuru (-3.0785)
Historical,genji (-3.5351),hijikata (-3.5351),mushishi (-3.5351),holo (-3.4830),tanjirou (-3.4830)
Kids,backkom (-4.2333),pero (-3.9184),shobu (-3.8882),mewtwo (-3.8562),team rocket (-3.8219)
Mecha,zeon (-4.2070),mobile suit (-4.1184),gunpla (-3.8001),universal century (-3.7147),nuku (-3.5719)
Music,hoshitani (-3.9103),kao council (-3.8672),ayanagi (-3.8200),kumiko (-3.7679),shower tv (-3.7097)
Mystery,mikiya (-4.0161),edogawa (-3.9362),conan edogawa (-3.8667),detective boy (-3.6894),kogorou mouri (-3.6894)


In [20]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, hamming_loss, jaccard_score, roc_auc_score
import numpy as np
import pandas as pd

def predict_genres_with_naives(classifier, vectorizer, df_source, mlb, id2label, threshold=0.5):
    # Vectorize clean synopsis
    X = vectorizer.transform(df_source['clean_synopsis'])
    y_true = mlb.transform(df_source['genres'])

    # Predict probabilities and apply threshold
    probabilities = classifier.predict_proba(X)
    probabilities = np.array(probabilities)  # list of arrays → array
    if isinstance(probabilities, list):  # OneVsRestClassifier returns list of arrays
        probabilities = np.vstack([p[:, 1] for p in probabilities]).T
    predictions = (probabilities >= threshold).astype(int)

    # Decode predicted genres using id2label
    predicted_genres = []
    for row in predictions:
        genres = [id2label[i] for i, val in enumerate(row) if val == 1]
        predicted_genres.append(genres)

    # === Calculate metrics ===
    f1 = f1_score(y_true, predictions, average='macro')
    precision = precision_score(y_true, predictions, average='macro')
    recall = recall_score(y_true, predictions, average='macro')
    accuracy = accuracy_score(y_true, predictions)
    hamming = hamming_loss(y_true, predictions)
    jaccard = jaccard_score(y_true, predictions, average='macro')
    hit_rate = (np.logical_and(y_true, predictions).sum(axis=1) > 0).mean()
    try:
        roc_auc = roc_auc_score(y_true, probabilities, average='macro')
    except ValueError:
        roc_auc = np.nan

    metrics_df = pd.DataFrame({
        'Metric': [
            'F1 Score', 'Precision', 'Recall', 'Exact Match Accuracy',
            'Hamming Loss', 'Jaccard Score', 'Hit Rate', 'ROC AUC'
        ],
        'Value': [
            f1, precision, recall, accuracy,
            hamming, jaccard, hit_rate, roc_auc
        ]
    })

    # Build result DataFrame
    result_df = pd.DataFrame({
        "synopsis": df_source["synopsis"].values,
        "true_genres": df_source["genres"].values,
        "predicted_genres": predicted_genres
    })

    print("\n📊 Evaluation Metrics:")
    print(metrics_df)

    return result_df, metrics_df, probabilities, y_true, predictions

# print("0.5 Thresh")
# use_thresh=0.5
# id2label = {i: label for i, label in enumerate(mlb.classes_)}
# #df_train_results, train_metrics, train_prob, train_labels, train_pred  = predict_genres_with_naives(classifier, vectorizer, df_train, mlb, id2label, threshold=use_thresh)
# df_val_results, val_metrics, val_prob, val_labels, val_pred = predict_genres_with_naives(classifier, vectorizer, df_val, mlb, id2label, threshold=use_thresh)
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)


# print("prior Thresh")
# use_thresh=prior_threshold
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

# print("F1 Thresh")
# optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
# use_thresh=optimised_thresh
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

# print("MCC Thresh")
# optimised_thresh=optimize_thresholds_with_mcc_constraint(val_prob,val_labels)
# use_thresh=optimised_thresh
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

In [21]:
use_thresh=0.5
id2label = {i: label for i, label in enumerate(mlb.classes_)}
#df_train_results, train_metrics, train_prob, train_labels, train_pred  = predict_genres_with_naives(classifier, vectorizer, df_train, mlb, id2label, threshold=use_thresh)
df_val_results, val_metrics, val_prob, val_labels, val_pred = predict_genres_with_naives(classifier, vectorizer, df_val, mlb, id2label, threshold=use_thresh)
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)


📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.540043
1             Precision  0.554933
2                Recall  0.540256
3  Exact Match Accuracy  0.098655
4          Hamming Loss  0.123038
5         Jaccard Score  0.379643
6              Hit Rate  0.848655
7               ROC AUC  0.821518

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.529547
1             Precision  0.549325
2                Recall  0.530849
3  Exact Match Accuracy  0.075196
4          Hamming Loss  0.127778
5         Jaccard Score  0.367733
6              Hit Rate  0.835017
7               ROC AUC  0.825164


In [10]:

from sklearn.metrics import classification_report

print(classification_report(train_labels, train_pred, target_names=all_genres))
print(classification_report(val_labels, val_pred, target_names=all_genres))
print(classification_report(test_labels, test_pred, target_names=all_genres))

               precision    recall  f1-score   support

       Action       0.77      0.82      0.79      2227
    Adventure       0.71      0.89      0.79      1552
       Comedy       0.81      0.88      0.84      2913
        Drama       0.77      0.87      0.82      1473
      Fantasy       0.83      0.82      0.83      1686
   Historical       0.67      0.97      0.79       529
         Kids       0.66      0.98      0.79       594
        Mecha       0.63      0.95      0.75       543
        Music       0.42      0.99      0.59       366
      Mystery       0.72      0.97      0.83       474
      Romance       0.72      0.85      0.78      1195
       School       0.63      0.93      0.75       968
       Sci-Fi       0.87      0.79      0.83      1419
       Seinen       0.97      0.81      0.88       507
       Shoujo       0.95      0.87      0.90       402
      Shounen       0.77      0.90      0.83      1141
Slice of Life       0.87      0.76      0.81      1004
       Sp

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
#display(df_train_results)
display(df_val_results)
display(df_test_results)

,synopsis,true_genres,predicted_genres
0,Follows a pig whose family's mission is to col...,"[Comedy, Kids]",[Kids]
1,The story begins 15 years after the demon king...,"[Comedy, Fantasy, Slice of Life]",[Fantasy]
2,"The story centers on Otarou, a creature themed...",[Comedy],[]
3,follows the adventures of an adorable jet plan...,"[Adventure, Kids]","[Adventure, Comedy, Kids]"
4,"An elite soldier, whose nickname is God of Dea...","[Action, Adventure, Comedy, Romance, Sci-Fi]","[Action, Comedy]"
...,...,...,...
887,This special is showing a day of Eclair's life...,"[Fantasy, Shounen]",[Comedy]
888,Episode 0 of . Kanami and Mihono are self-awar...,[Action],"[Comedy, Slice of Life]"
889,The ad's crossover theme matches a tote bag bu...,"[Seinen, Slice of Life]",[Comedy]
890,A girl's classmate disappears during a game of...,"[Shoujo, Supernatural]",[Comedy]


,synopsis,true_genres,predicted_genres
0,The daily life of the Motsumoto family. The th...,"[Comedy, Slice of Life]","[Comedy, Romance, Slice of Life]"
1,"The Soul Tree, the great source of our race. ...","[Action, Adventure, Kids]","[Action, Adventure, Fantasy]"
2,"Fairies living in a fluffy forest, where both ...","[Fantasy, Kids]","[Comedy, Fantasy, Kids, Shoujo, Slice of Life]"
3,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]","[Action, Adventure, Mecha, Sci-Fi, Shounen]"
4,The 2018 LINE sticker set Poccolies is inspiri...,"[Kids, Slice of Life]","[Comedy, Slice of Life]"
...,...,...,...
886,A series of four animated prequels to Street F...,[Action],"[Action, Shounen]"
887,"According to HMV, the DVD & Blu-ray volume 7 o...","[Comedy, Seinen, Sports]",[Comedy]
888,"Lucius ends up in a space station, where he ex...","[Comedy, Seinen]","[Comedy, Historical, Sci-Fi, Seinen]"
889,"The ""Megumi and Taiyou"" commercial depicts the...","[Romance, Slice of Life]","[Romance, School, Slice of Life]"


In [5]:
roc_auc_per_genre = roc_auc_score(test_labels, test_prob, average=None)
for label, auc in zip(all_genres, roc_auc_per_genre):
    print(f"{label}: ROC AUC = {auc:.4f}")


Action: ROC AUC = 0.8337
Adventure: ROC AUC = 0.8153
Comedy: ROC AUC = 0.7607
Drama: ROC AUC = 0.7251
Fantasy: ROC AUC = 0.8553
Historical: ROC AUC = 0.9093
Kids: ROC AUC = 0.8917
Mecha: ROC AUC = 0.8908
Music: ROC AUC = 0.8704
Mystery: ROC AUC = 0.8270
Romance: ROC AUC = 0.7895
School: ROC AUC = 0.8460
Sci-Fi: ROC AUC = 0.8797
Seinen: ROC AUC = 0.7240
Shoujo: ROC AUC = 0.7602
Shounen: ROC AUC = 0.6952
Slice of Life: ROC AUC = 0.8568
Sports: ROC AUC = 0.9594
Super Power: ROC AUC = 0.7763
Supernatural: ROC AUC = 0.8366


In [7]:
df=pd.DataFrame(roc_auc_per_genre)
display(df)
df.to_csv('MNB.csv', float_format='%.2f', index=False)


,0
0,0.833717
1,0.815253
2,0.760702
3,0.725094
4,0.855291
5,0.909275
6,0.891743
7,0.890784
8,0.870414
9,0.827045


In [ ]:
from sklearn.metrics import matthews_corrcoef
import numpy as np
import pandas as pd

def mcc_per_label(y_true, y_pred, label_names=None):
    n_labels = y_true.shape[1]
    mcc_scores = []

    for i in range(n_labels):
        mcc = matthews_corrcoef(y_true[:, i], y_pred[:, i])
        mcc_scores.append(mcc)

    if label_names is None:
        label_names = [f"Label_{i}" for i in range(n_labels)]

    return pd.Series(mcc_scores, index=label_names, name="MCC")

MCC_labels=mcc_per_label(val_labels,val_pred,label_names=all_genres)
display(MCC_labels)

In [1]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

In [ ]:
#Instead of using a fixed threshold we compare it to the prior threshold from the train set
def predict_using_prior_threshold(X, classifier, priors):
    """
    Predicts genres using P(c|x) > P(c) as threshold logic.
    - classifier: trained OneVsRestClassifier
    - priors: array of prior probabilities (i.e., class proportions in y_train)
    """
    probs = classifier.predict_proba(X)
    # compare each prob with its class's prior
    preds = (probs > priors).astype(int)
    return preds





In [1]:
#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting
from sklearn.metrics import hamming_loss

# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
priors = y_train.mean(axis=0)
#display(y)
vectorizer = TfidfVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4))
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

classifier = OneVsRestClassifier(MultinomialNB(alpha=0.1))#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)                                    #At least when it is not balanced it can overfit the train class

# from sklearn.metrics import classification_report
#samples	How well did we predict each movie's genres?	✅ Best for your task
#micro	    Overall genre prediction accuracy	            ✅ Good for global view
#macro	    Equal focus on rare and frequent genres	        ⚠️ Careful with imbalance
#weighted	Genre performance balanced by frequency	        ✅ Fair but skew-sensitive
#USE SAMPLES


print("Report for Train dataset")
#threshold=0.08
#y_pred_train=LR_OVA_prob(X_train,threshold,classifier)
y_pred_train = predict_using_prior_threshold(X_train, classifier, priors)
print_report(y_train,y_pred_train,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_train, y_pred_train, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_train, y_pred_train)
print("Hit Rate:", hr)
hl = hamming_loss(y_train, y_pred_train)
print("Hamming Loss:", hl)

print("----------------------")
print("Report for Val dataset")
#y_pred_val=LR_OVA_prob(X_val,threshold,classifier)
y_pred_val = predict_using_prior_threshold(X_val, classifier, priors)
print_report(y_val,y_pred_val,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_val, y_pred_val, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)
hl = hamming_loss(y_val, y_pred_val)
print("Hamming Loss:", hl)


print("----------------------")
print("Report for Test dataset")
#y_pred_test=LR_OVA_prob(X_test,threshold,classifier)
y_pred_test = predict_using_prior_threshold(X_test, classifier, priors)
print_report(y_test,y_pred_test,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_test, y_pred_test, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)
hl = hamming_loss(y_test, y_pred_test)
print("Hamming Loss:", hl)



display(result_table(y_train,y_pred_train,df_train,mlb))
display(result_table(y_val,y_pred_val,df_val,mlb))
display(result_table(y_test,y_pred_test,df_test,mlb))

NameError: name 'MultiLabelBinarizer' is not defined